In [81]:
# import the libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder


In [58]:
#data = pd.read_csv("marks.csv")
data = pd.read_csv('synth_composite.csv')


In [59]:
len(data)

6362620

In [124]:
df = data.sample(frac=0.10)
df.head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
3957525,540,CASH_OUT,240721.258262,160_6110462_0,3.394469e-02,0.000000e+00,160_13965_0,1.158615e+07,1.182709e+07,0,0
105696,401,CASH_IN,301971.710563,160_5806012_0,4.424726e+06,4.122755e+06,160_259992_0,4.743223e+06,4.441111e+06,0,0
1374470,358,PAYMENT,27709.911111,160_5163286_0,1.535749e+04,0.000000e+00,160_2205806_0,9.693117e+00,1.659299e+01,0,0


In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 636262 entries, 3957525 to 6329837
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   step            636262 non-null  int64  
 1   type            636262 non-null  object 
 2   amount          636262 non-null  float64
 3   nameOrig        636262 non-null  object 
 4   oldbalanceOrg   636262 non-null  float64
 5   newbalanceOrig  636262 non-null  float64
 6   nameDest        636262 non-null  object 
 7   oldbalanceDest  636262 non-null  float64
 8   newbalanceDest  636262 non-null  float64
 9   isFraud         636262 non-null  int64  
 10  isFlaggedFraud  636262 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 58.3+ MB


We have some fetures of object type and no missing values

In [126]:
cateorical_features = [features for features in df.columns if df[features].dtypes == 'O']
cateorical_features

['type', 'nameOrig', 'nameDest']

In [127]:
# we use label encoder to convert the categorical variables to numerical values
le = LabelEncoder()
for feature in cateorical_features:
    df[feature] = le.fit_transform(df[feature])
df.head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
3957525,540,1,240721.258262,568623,3.394469e-02,0.000000e+00,78393,1.158615e+07,1.182709e+07,0,0
105696,401,0,301971.710563,534804,4.424726e+06,4.122755e+06,266072,4.743223e+06,4.441111e+06,0,0
1374470,358,3,27709.911111,463708,1.535749e+04,0.000000e+00,213630,9.693117e+00,1.659299e+01,0,0


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 636262 entries, 3957525 to 6329837
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   step            636262 non-null  int64  
 1   type            636262 non-null  int64  
 2   amount          636262 non-null  float64
 3   nameOrig        636262 non-null  int64  
 4   oldbalanceOrg   636262 non-null  float64
 5   newbalanceOrig  636262 non-null  float64
 6   nameDest        636262 non-null  int64  
 7   oldbalanceDest  636262 non-null  float64
 8   newbalanceDest  636262 non-null  float64
 9   isFraud         636262 non-null  int64  
 10  isFlaggedFraud  636262 non-null  int64  
dtypes: float64(5), int64(6)
memory usage: 58.3 MB


In [142]:
# defining and fitting the model
random_state = np.random.RandomState(42)
model = IsolationForest(n_estimators = 100,
                       max_samples = 'auto',
                       contamination = 0.001,
                       random_state = random_state)
model.fit(df)

IsolationForest(contamination=0.001,
                random_state=RandomState(MT19937) at 0x7FEF0AF8ED40)

In [143]:
print (model.get_params)

<bound method BaseEstimator.get_params of IsolationForest(contamination=0.001,
                random_state=RandomState(MT19937) at 0x7FEF0AF8ED40)>


In [144]:
# getting the results

df['anomaly_score'] = model.predict(df)

In [145]:
list(df.anomaly_score).count(-1)

637

In [146]:
df.anomaly_score = df.anomaly_score.apply(lambda x: 1 if x == -1 else 0)
#comb["Large_Family"] = comb.Family.map(lambda x: 1 if x >4  else 0)


In [147]:
df.anomaly_score.unique()

array([0, 1])

In [149]:
 list(df.anomaly_score).count(1)

637

In [152]:
df[(df.anomaly_score==1) & (df.isFraud==1)].count()

step              6
type              6
amount            6
nameOrig          6
oldbalanceOrg     6
newbalanceOrig    6
nameDest          6
oldbalanceDest    6
newbalanceDest    6
isFraud           6
isFlaggedFraud    6
anomaly_score     6
dtype: int64

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 636262 entries, 3957525 to 6329837
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   step            636262 non-null  int64  
 1   type            636262 non-null  int64  
 2   amount          636262 non-null  float64
 3   nameOrig        636262 non-null  int64  
 4   oldbalanceOrg   636262 non-null  float64
 5   newbalanceOrig  636262 non-null  float64
 6   nameDest        636262 non-null  int64  
 7   oldbalanceDest  636262 non-null  float64
 8   newbalanceDest  636262 non-null  float64
 9   isFraud         636262 non-null  int64  
 10  isFlaggedFraud  636262 non-null  int64  
 11  anomaly_score   636262 non-null  int64  
dtypes: float64(5), int64(7)
memory usage: 63.1 MB
